# Processing FIB data for the Safe to Swim map (v2)

### Introduction
The following code processes fecal indicator bacteria data (FIB) for the Safe to Swim map (v2), which is currently in development. It sources FIB data from the [BeachWatch](https://beachwatch.waterboards.ca.gov/) and [California Environmental Data Exchange Network (CEDEN)](https://ceden.org/) databases, both of which are managed by the [State Water Resources Control Board](https://www.waterboards.ca.gov/). It also sources data from the [Lower American River Recreational Water Quality Web App](https://experience.arcgis.com/experience/47e27f245e044ac2a8e15083307e75f6/?draft=true&org=waterboards), which is managed by the Central Valley Regional Water Board. This script integrates the three datasets and calculates the rolling 30-day and 6-week geometric mean values for each sample data point. The FIB data used in this script includes sampling data for *E. coli*, Enterococcus, Fecal Coliform, and Total Coliform.

### Requirements
To run the following code, you will need Python 3.x installed along with the Python packages, pandas and pyodbc. You will also need access to the internal BeachWatch and CEDEN data tables via internal data mart or some other access point.

### Instructions
Run the following code cells in sequential order. You can run them manually cell by cell or run them all in one go. Do not skip any steps or cells. Depending on your computer and/or internet connection, it can take around two hours to run the script in its entirety. The generated data files are saved in the main directory.

### 1. Import the required Python packages

In [1]:
from datetime import datetime, timedelta
import numpy as np
import os
import pandas as pd
import pyodbc # Used for connecting to the internal data marts
import requests
from scipy.stats.mstats import gmean

### 2. Download FIB data from BeachWatch, CEDEN, and the Lower American River E. coli map

#### 2.1 BeachWatch
Define the variables for connecting to BeachWatch. These are private login credentials. The code block below will not run unless the environment variables on your machine are set up similarly.

In [2]:
BW_SERVER1 = os.environ.get('S2S_Server')
BW_DATABASE = os.environ.get('S2S_DB')
BW_TABLE = os.environ.get('S2S_Table')
BW_UID = os.environ.get('S2S_User')
BW_PWD = os.environ.get('S2S_Pass')

Define and run a function for connecting to BeachWatch, querying all data records from BeachWatch, and returning the data as a pandas dataframe.

In [3]:
# Define the date columns for both BeachWatch and CEDEN to ensure that date values get parsed correctly
date_cols = ['SampleDate', 'CalibrationDate', 'CollectionTime', 'PrepPreservationDate', 'DigestExtractDate', 'AnalysisDate']

def get_bw_data():
    cnxn = pyodbc.connect(Driver='SQL Server', Server=BW_SERVER1, Database=BW_DATABASE, uid=BW_UID, pwd=BW_PWD)
    sql =  "SELECT * FROM %s" % BW_TABLE
    df = pd.read_sql_query(sql, cnxn, parse_dates=date_cols, dtype={'Result': np.float64, 'ResultReplicate': np.int16, 'CollectionReplicate': np.int16})
    return df

bw_df = get_bw_data() 
print("Count of rows:", bw_df.shape[0])

# Add a field for identifying the database source of the data
bw_df['DataSource'] = 'BeachWatch'

pd.set_option('display.max_columns', None)
bw_df.head()

C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\1589059262.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, cnxn, parse_dates=date_cols, dtype={'Result': np.float64, 'ResultReplicate': np.int16, 'CollectionReplicate': np.int16})


Count of rows: 2257976


,ProgramName,ParentProjectName,ProjectName,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,UnitName,Result,Observation,MDL,RL,ResQualCode,QACode,BatchVerificationCode,ComplianceCode,SampleComments,LabCollectionComments,LabResultComments,BatchComments,EventCode,ProtocolCode,AgencyCode,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceName,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,LabSubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,None,0,0,<,NR,NR,NR,None,None,None,,wq,Not Recorded,CLBHHS,None,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Recorded,"Coliform, Total",BeachWatch
1,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2011-09-12,1900-01-01 07:00:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-09/12/2011,Not Recorded,samplewater,Colilert-18,E. coli,MPN/100 mL,10.0,None,0,0,<,NR,NR,NR,None,None,None,,wq,Not Recorded,CLBHHS,None,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Recorded,E. coli,BeachWatch
2,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2011-09-26,1900-01-01 07:00:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-09/26/2011,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,None,0,0,<,NR,NR,NR,None,None,None,,wq,Not Recorded,CLBHHS,None,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Recorded,"Coliform, Total",BeachWatch
3,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,E. coli,MPN/100 mL,10.0,None,0,0,<,NR,NR,NR,None,None,None,,wq,Not Recorded,CLBHHS,None,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Recorded,E. coli,BeachWatch
4,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-21,1900-01-01 10:40:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/21/2015,Not Recorded,samplewater,Enterolert,Enterococcus,MPN/100 mL,10.0,None,0,0,=,NR,NR,NR,None,None,None,,wq,Not Recorded,CLBHHS,None,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Recorded,Enterococcus,BeachWatch


Some of the BeachWatch columns have slightly different names compared to the CEDEN columns. Because we will be joining these two datasets, we want all of the column names to match.

In [4]:
# Dictionary for mapping the names of BeachWatch fields to CEDEN fields
bw_to_ceden_fields = {
    'ProgramName': 'Program',
    'ParentProjectName': 'ParentProject',
    'ProjectName': 'Project',
    'UnitName': 'Unit',
    'ResQualCode': 'ResultQualCode',
    'BatchVerificationCode': 'BatchVerification',
    'LabCollectionComments': 'CollectionComments',
    'LabResultComments': 'ResultsComments',
    'AgencyCode': 'SampleAgency',
    'CollectionDeviceName': 'CollectionDeviceDescription',
    'LabSubmissionCode': 'SubmissionCode',
    'ResultReplicate': 'ResultsReplicate'
}

bw_df = bw_df.rename(columns=bw_to_ceden_fields)
bw_df.head()

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,None,0,0,<,NR,NR,NR,None,None,None,,wq,Not Recorded,CLBHHS,None,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Recorded,"Coliform, Total",BeachWatch
1,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2011-09-12,1900-01-01 07:00:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-09/12/2011,Not Recorded,samplewater,Colilert-18,E. coli,MPN/100 mL,10.0,None,0,0,<,NR,NR,NR,None,None,None,,wq,Not Recorded,CLBHHS,None,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Recorded,E. coli,BeachWatch
2,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2011-09-26,1900-01-01 07:00:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-09/26/2011,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,None,0,0,<,NR,NR,NR,None,None,None,,wq,Not Recorded,CLBHHS,None,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Recorded,"Coliform, Total",BeachWatch
3,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,E. coli,MPN/100 mL,10.0,None,0,0,<,NR,NR,NR,None,None,None,,wq,Not Recorded,CLBHHS,None,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Recorded,E. coli,BeachWatch
4,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-21,1900-01-01 10:40:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/21/2015,Not Recorded,samplewater,Enterolert,Enterococcus,MPN/100 mL,10.0,None,0,0,=,NR,NR,NR,None,None,None,,wq,Not Recorded,CLBHHS,None,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Not Recorded,Enterococcus,BeachWatch


#### 2.2 CEDEN
Define the variables for connecting to CEDEN. Like for the BeachWatch data above, these are private login credentials. 

In [5]:
CEDEN_SERVER1 = os.environ.get('SERVER1')
CEDEN_UID = os.environ.get('UID')
CEDEN_PWD = os.environ.get('PWD')
CEDEN_TABLE = os.environ.get('TABLE')
CEDEN_SITE_DATUM_TABLE = os.environ.get('SITE_DATUM_TABLE') # Used for getting site datum data
CEDEN_SITE_TABLE = os.environ.get('SITE_TABLE') # Used for getting site region number

Define and run a function for connecting to the CEDEN data mart and returning the data as a pandas dataframe. This query includes all data for E. coli, Enterococcus, Fecal Coliform, and Total Coliform, but at the same time it excludes all records where Program == BeachWatch. There is a lot of duplicate BeachWatch data in CEDEN from the time when BeachWatch data was copied over into CEDEN. We want to exclude the duplicate BeachWatch data from our query.

In [6]:
def get_ceden_data():
    cnxn = pyodbc.connect(Driver='SQL Server', Server=CEDEN_SERVER1, uid=CEDEN_UID, pwd=CEDEN_PWD)
    sql = "SELECT * FROM %s WHERE (Analyte in ('E. coli', 'Enterococcus', 'Coliform, Total', 'Coliform, Fecal') AND Program != 'BeachWatch')" % CEDEN_TABLE
    df = pd.read_sql_query(sql, cnxn, parse_dates=date_cols, dtype={'Result': np.float64, 'ResultsReplicate': np.int16, 'CollectionReplicate': np.int16})
    return df

ceden_df = get_ceden_data()
print("Count of rows:", ceden_df.shape[0])

# Add data source field
ceden_df['DataSource'] = 'CEDEN'

ceden_df.head()

C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\1970532632.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, cnxn, parse_dates=date_cols, dtype={'Result': np.float64, 'ResultsReplicate': np.int16, 'CollectionReplicate': np.int16})


Count of rows: 393069


,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,HydroMod,HydroModLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,DataSource
0,OCSD Core Ocean Monitoring Program,OCSD Core Ocean Monitoring Program,OCSD Core Ocean Monitoring Program,OCSD Ocean WQ Station 2212,2212,2012-08-09,1899-12-30 08:22:00,Not Recorded,14,m,Not Recorded,1,1,None,None,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,None,None,None,=,None,NR,NR,None,,,None,WQ,Not Recorded,OCSD,None,Field Method,33.62017,-117.98087,Not Recorded,1899-12-30,Subsurface,None,NaT,None,NaT,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"Coliform, Total",False,OCSD_COMP,OCSD_COMP,OCSD_COMP,5,1129,None,0,CEDEN
1,OCSD Core Ocean Monitoring Program,OCSD Core Ocean Monitoring Program,OCSD Core Ocean Monitoring Program,OCSD Ocean WQ Station 2213,2213,2012-08-09,1899-12-30 10:08:00,Not Recorded,10,m,Not Recorded,1,1,None,None,samplewater,Enterolert,Enterococcus,MPN/100 mL,10.0,None,None,None,=,None,NR,NR,None,,,None,WQ,Not Recorded,OCSD,None,Field Method,33.61032,-117.98905,Not Recorded,1899-12-30,Subsurface,None,NaT,None,NaT,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Enterococcus,False,OCSD_COMP,OCSD_COMP,OCSD_COMP,5,635,None,0,CEDEN
2,OCSD Core Ocean Monitoring Program,OCSD Core Ocean Monitoring Program,OCSD Core Ocean Monitoring Program,OCSD Ocean WQ Station 2220,2220,2012-08-09,1899-12-30 08:43:00,Not Recorded,1,m,Not Recorded,1,1,None,None,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,None,None,None,=,None,NR,NR,None,,,None,WQ,Not Recorded,OCSD,None,Field Method,33.63658,-117.98180,Not Recorded,1899-12-30,Subsurface,None,NaT,None,NaT,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"Coliform, Total",False,OCSD_COMP,OCSD_COMP,OCSD_COMP,5,1129,None,0,CEDEN
3,OCSD Core Ocean Monitoring Program,OCSD Core Ocean Monitoring Program,OCSD Core Ocean Monitoring Program,OCSD Ocean WQ Station 2222,2222,2012-08-09,1899-12-30 09:01:00,Not Recorded,1,m,Not Recorded,1,1,None,None,samplewater,Enterolert,Enterococcus,MPN/100 mL,10.0,None,None,None,=,None,NR,NR,None,,,None,WQ,Not Recorded,OCSD,None,Field Method,33.62537,-117.98957,Not Recorded,1899-12-30,Subsurface,None,NaT,None,NaT,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Enterococcus,False,OCSD_COMP,OCSD_COMP,OCSD_COMP,5,635,None,0,CEDEN
4,OCSD Core Ocean Monitoring Program,OCSD Core Ocean Monitoring Program,OCSD Core Ocean Monitoring Program,OCSD Ocean WQ Station 2222,2222,2012-08-09,1899-12-30 09:01:00,Not Recorded,10,m,Not Recorded,1,1,None,None,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,None,None,None,=,None,NR,NR,None,,,None,WQ,Not Recorded,OCSD,None,Field Method,33.62537,-117.98957,Not Recorded,1899-12-30,Subsurface,None,NaT,None,NaT,NaT,None,None,None,None,None,None,None,None

#### 2.3 Central Valley Regional Water Board - Lower American River *E. coli* Monitoring Results
Get the Region 5 data from the open data portal (https://data.ca.gov/dataset/central-valley-water-board-e-coli-monitoring-results) and transform it to the BeachWatch/CEDEN format.

In [7]:
# Use the open data portal API to fetch the data
cv_url = 'https://data.ca.gov/api/3/action/datastore_search?resource_id=fc450fb6-e997-4bcf-b824-1b3ed0f06045&limit=10000'
cv_response = requests.get(cv_url)
cv_text = cv_response.json()['result']['records']
cv_df = pd.DataFrame(pd.json_normalize(cv_text))

print("Count of rows:", cv_df.shape[0])
cv_df.head()

Count of rows: 1873


,_id,StationCode,StationName,Latitude,Longitude,SampleDate,Analyte,Result,Unit,Program
0,1,519LSAC52,American River at Lower Sunrise Area,38.633,-121.271,2025-06-10,E. coli,30.1,MPN/100 mL,Surface Water Ambient Monitoring Program
1,2,519LSAC52,American River at Lower Sunrise Area,38.633,-121.271,2025-06-03,E. coli,43.5,MPN/100 mL,Surface Water Ambient Monitoring Program
2,3,519LSAC52,American River at Lower Sunrise Area,38.633,-121.271,2025-05-27,E. coli,90.6,MPN/100 mL,Surface Water Ambient Monitoring Program
3,4,519LSAC52,American River at Lower Sunrise Area,38.633,-121.271,2025-05-20,E. coli,50.4,MPN/100 mL,Surface Water Ambient Monitoring Program
4,5,519LSAC52,American River at Lower Sunrise Area,38.633,-121.271,2025-05-13,E. coli,44.8,MPN/100 mL,Surface Water Ambient Monitoring Program


In [8]:
# Drop ID column
cv_df = cv_df.drop('_id', axis=1)

# Rename columns to match CEDEN format
cv_df = cv_df.rename(columns={'Latitude': 'TargetLatitude', 'Longitude': 'TargetLongitude'})

# Define data types
cv_df['SampleDate'] = pd.to_datetime(cv_df['SampleDate'])
cv_df['TargetLatitude'] = cv_df['TargetLatitude'].astype(float)
cv_df['TargetLongitude'] = cv_df['TargetLongitude'].astype(float)

# Add replicate fields
cv_df['CollectionReplicate'] = 1
cv_df['ResultsReplicate'] = 1

# Add matrix field
cv_df['MatrixName'] = 'samplewater'

# Add data source field
cv_df['DataSource'] = 'Central Valley Water Board'

cv_df.head()

,StationCode,StationName,TargetLatitude,TargetLongitude,SampleDate,Analyte,Result,Unit,Program,CollectionReplicate,ResultsReplicate,MatrixName,DataSource
0,519LSAC52,American River at Lower Sunrise Area,38.633,-121.271,2025-06-10,E. coli,30.1,MPN/100 mL,Surface Water Ambient Monitoring Program,1,1,samplewater,Central Valley Water Board
1,519LSAC52,American River at Lower Sunrise Area,38.633,-121.271,2025-06-03,E. coli,43.5,MPN/100 mL,Surface Water Ambient Monitoring Program,1,1,samplewater,Central Valley Water Board
2,519LSAC52,American River at Lower Sunrise Area,38.633,-121.271,2025-05-27,E. coli,90.6,MPN/100 mL,Surface Water Ambient Monitoring Program,1,1,samplewater,Central Valley Water Board
3,519LSAC52,American River at Lower Sunrise Area,38.633,-121.271,2025-05-20,E. coli,50.4,MPN/100 mL,Surface Water Ambient Monitoring Program,1,1,samplewater,Central Valley Water Board
4,519LSAC52,American River at Lower Sunrise Area,38.633,-121.271,2025-05-13,E. coli,44.8,MPN/100 mL,Surface Water Ambient Monitoring Program,1,1,samplewater,Central Valley Water Board


### 3. Combine the BeachWatch, CEDEN, and R5 datasets
The BeachWatch and CEDEN datasets have similar data structures, allowing us to combine the two datasets and work on both of them at the same time. The R5 dataset is missing a lot of columns but should still combine with the other two datasets without issue.

In [9]:
combined_df = pd.concat([bw_df, ceden_df, cv_df],  ignore_index=True)
print("Count of rows:", combined_df.shape[0])

# Fill NaN values in the Central Valley records with None
#combined_df.loc[combined_df['DataSource'] == 'Central Valley Water Board'].fillna('', inplace=True)
combined_df.fillna("", inplace=True)

combined_df.tail()

Count of rows: 2652918


C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\1504047934.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combined_df.fillna("", inplace=True)


,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode
2652913,Surface Water Ambient Monitoring Program,,,American River at Tiscornia Beach,519AMNSAC,2018-02-08,NaT,,,,,1,1,,,samplewater,,E. coli,MPN/100 mL,579.4,,,,,,,,,,,,,,,,,38.598,-121.507,,NaT,,,NaT,,NaT,NaT,,,,,,,,,,,,,,,,,,,,,,,,Central Valley Water Board,,,,,,,,,,
2652914,Surface Water Ambient Monitoring Program,,,American River at Tiscornia Beach,519AMNSAC,2018-01-30,NaT,,,,,1,1,,,samplewater,,E. coli,MPN/100 mL,1203.3,,,,,,,,,,,,,,,,,38.598,-121.507,,NaT,,,NaT,,NaT,NaT,,,,,,,,,,,,,,,,,,,,,,,,Central Valley Water Board,,,,,,,,,,
2652915,Surface Water Ambient Monitoring Program,,,American River at Tiscornia Beach,519AMNSAC,2018-01-24,NaT,,,,,1,1,,,samplewater,,E. coli,MPN/100 mL,488.4,,,,,,,,,,,,,,,,,38.598,-121.507,,NaT,,,NaT,,NaT,NaT,,,,,,,,,,,,,,,,,,,,,,,,Central Valley Water Board,,,,,,,,,,
2652916,Surface Water Ambient Monitoring Program,,,American River at Tiscornia Beach,519AMNSAC,2018-01-18,NaT,,,,,1,1,,,samplewater,,E. coli,MPN/100 mL,648.8,,,,,,,,,,,,,,,,,38.598,-121.507,,NaT,,,NaT,,NaT,NaT,,,,,,,,,,,,,,,,,,,,,,,,Central Valley Water Board,,,,,,,,,,
2652917,Surface Water Ambient Monitoring Program,,,American River at Tiscornia Beach,519AMNSAC,2018-01-11,NaT,,,,,1,1,,,samplewater,,E. coli,MPN/100 mL,344.8,,,,,,,,,,,,,,,,,38.598,-121.507,,NaT,,,NaT,,NaT,NaT,,,,,,,,,,,,,,,,,,,,,,,,Central Valley Water Board,,,,,,,,,,


### 4. Create the SampleDateTime column
For CEDEN, the sample date and collection time are stored in two different columns, SampleDate and CollectionTime, respectively. CollectionTime has a recorded date along with a time, but the paired date is not usable. Create a new column by separating out the time value from the CollectionTime column and combine it with the date value in the SampleDate column.

In [10]:
# Extract the time value from CollectionTime field and copy to a new field
combined_df['CollectionTimeOnly'] = combined_df['CollectionTime'].dt.time

# If the extracted time value is null or NaT, replace the empty value with 00:00:00
combined_df['CollectionTimeOnly'] = combined_df['CollectionTimeOnly'].fillna(pd.Timestamp('2025-01-01T00').time())

# Combine the date and time values into a new SampleDateTime field
combined_df['SampleDateTime'] = pd.to_datetime(combined_df['SampleDate']) + pd.to_timedelta(combined_df['CollectionTimeOnly'].astype(str))

combined_df.head()

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,SampleDateTime
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,,0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:08:00,2015-07-07 11:08:00
1,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2011-09-12,1900-01-01 07:00:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-09/12/2011,Not Recorded,samplewater,Colilert-18,E. coli,MPN/100 mL,10.0,,0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,E. coli,BeachWatch,,,,,,,,,,,07:00:00,2011-09-12 07:00:00
2,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2011-09-26,1900-01-01 07:00:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-09/26/2011,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,,0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,07:00:00,2011-09-26 07:00:00
3,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,E. coli,MPN/100 mL,10.0,,0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,E. coli,BeachWatch,,,,,,,,,,,11:08:00,2015-07-07 11:08:00
4,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-21,1900-01-01 10:40:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/21/2015,Not Recorded,samplewater,Enterolert,Enterococcus,MPN/100 mL,10.0,,0,0,=,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,Enterococcus,BeachWatch,,,,,,,,,,,10:40:00,2015-07-21 10:40:00


### 5. Dropping duplicate records
Even though we excluded BeachWatch records when pulling data from CEDEN (Step 2.2), there are still some duplicate BeachWatch records in CEDEN because these records are submitted to CEDEN under a different program name (i.e., not BeachWatch). 

An example of this is StationCode == 'Wharf-East' for Total coliform, sample taken on 9/5/2019. There are three data points for the same result, one in BeachWatch and two in CEDEN. They mostly have the same values in every column except for Program, ResultQualCode, and QACode. The Program value in BeachWatch is "BeachWatch" whereas the Program values in CEDEN are "BeachWatch" and "Santa Cruz City Environmental Program". The BeachWatch record was copied over into CEDEN from the BeachWatch database, and the other record was submitted to CEDEN under a different program name. Because the SQL query used in Step 2.2 only excludes records that have a Program value of "BeachWatch", the latter record would still make it into the combined dataset.

A list of columns, defined below in the variable "duplicate_cols", is used to identify and drop the remaining duplicate records. When comparing one record to another, the code is looking for at least one unique value across all of these columns. If the values for both records across all columns are the same, then it is considered a duplicate record. This list of columns can be changed, as needed.

In [11]:
# Sort the dataframe by the DataSource column so that all BeachWatch records are positioned before the CEDEN records. 
# This is to ensure that BeachWatch records are kept by default if there happens to be the same record from both BeachWatch and CEDEN
combined_df = combined_df.sort_values(by='DataSource')

# Convert Result field to numeric before removing duplicates. Duplicated and drop_duplicates don't work properly without the type conversion
# Ex. 519SAC104, E. coli, 10/13/20 CEDN + R5
combined_df['Result'] = pd.to_numeric(combined_df['Result'])

combined_df.head()

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,SampleDateTime
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,,0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:08:00,2015-07-07 11:08:00
1505327,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-16,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-16 00:00:00
1505326,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-09,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/09/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-09 00:00:00
1505325,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-12-09,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-12/09/2024,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,60.0,,10,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:27:00,2024-12-09 11:27:00
1505324,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-10-28,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-10/28/2024,Not Recorded,samplewater,Colilert-18,E. coli,MPN/100 mL,41.0,,10,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,E. coli,BeachWatch,,,,,,,,,,,11:27:00,2024-10-28 11:27:00


In [12]:
# Define the columns used to identify duplicate records
# 10/1/24 - I removed 'QACode' and 'ResultQualCode' from this list because it appears that some duplicate records across BeachWatch and CEDEN have different QACode and ResultQualCode values 
# See StationCode == 'Wharf-East' for Total coliform, samples taken on 9/5/2019 (QACode) and 9/23/2019 (ResultQualCode)
# 2/21/25 - I changed SampleDateTime to SampleDate and removed MethodName. This is to address issue with there being duplicate records between the R5 data and CEDEN. Ex. 519SAC104, E. coli, 10/13/2020
duplicate_cols = ['StationCode', 'Analyte', 'MatrixName', 'SampleDateTime', 'CollectionReplicate', 'ResultsReplicate', 'Result', 'Unit']

# Select the identified duplicate records from the combined dataset and copy them to a new dataframe
# These records will later be added to the rejected_records csv file output
duplicates_df = combined_df.loc[combined_df.duplicated(subset=duplicate_cols, keep='first')]
duplicates_df['Comments'] = 'Duplicate record'

print('Count of duplicate records:', duplicates_df.shape[0])
duplicates_df.head() 

Count of duplicate records: 32941


C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\3348846028.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates_df['Comments'] = 'Duplicate record'


,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,SampleDateTime,Comments
1467160,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,PL-050-non-accessible or restricted access sho...,PL-050,2015-07-16,1900-01-01 10:41:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2015,Not Recorded,samplewater,SM 9222 B,"Coliform, Total",MPN/100 mL,20.0,,2,2,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.6794,-117.248,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,10:41:00,2015-07-16 10:41:00,Duplicate record
1473841,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"PL-080-Ocean Beach, San Diego",PL-080,2015-07-22,1900-01-01 08:38:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/22/2015,Not Recorded,samplewater,SM 9222 B,"Coliform, Total",MPN/100 mL,60.0,,2,2,=,J,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7326,-117.257,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,08:38:00,2015-07-22 08:38:00,Duplicate record
1476709,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"PL-090-Ocean Beach, San Diego",PL-090,2015-07-16,1900-01-01 08:59:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2015,Not Recorded,samplewater,SM 9222 B,"Coliform, Fecal",MPN/100 mL,8.0,,2,2,=,J,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7467,-117.255,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Fecal",BeachWatch,,,,,,,,,,,08:59:00,2015-07-16 08:59:00,Duplicate record
1463793,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,PL-040-non-accessible or restricted access sho...,PL-040,2015-07-16,1900-01-01 10:22:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2015,Not Recorded,samplewater,SM 9222 B,"Coliform, Fecal",MPN/100 mL,2.0,,2,2,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.6653,-117.243,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Fecal",BeachWatch,,,,,,,,,,,10:22:00,2015-07-16 10:22:00,Duplicate record
1463794,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,PL-040-non-accessible or restricted access sho...,PL-040,2015-07-22,1900-01-01 09:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/22/2015,Not Recorded,samplewater,SM 9222 B,Enterococcus,MPN/100 mL,2.0,,2,2,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.6653,-117.243,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,Enterococcus,BeachWatch,,,,,,,,,,,09:00:00,2015-07-22 09:00:00,Duplicate record


In [13]:
print('Count of rows before dropping duplicates:', combined_df.shape[0])

# Drop the duplicate records from the combined dataset; if there are duplicates, keep the first duplicate record found (BeachWatch)
combined_df = combined_df.drop_duplicates(subset=duplicate_cols, keep='first')

print('Count of rows after removing duplicates:', combined_df.shape[0])

Count of rows before dropping duplicates: 2652918
Count of rows after removing duplicates: 2619977


### 6. Clean and process data

#### 6.1 Strip special characters and whitespace characters. Check null/missing values for compatability with the open data portal (data.ca.gov).

In [14]:
# Strip special characters. These characters can cause issues when reading, parsing, or writing the data
combined_df.replace(r'\t',' ', regex=True, inplace=True) # tab
combined_df.replace(r'\r',' ', regex=True, inplace=True) # carriage return
combined_df.replace(r'\n',' ', regex=True, inplace=True) # newline
combined_df.replace(r'\f',' ', regex=True, inplace=True) # formfeed
combined_df.replace(r'\v',' ', regex=True, inplace=True) # vertical tab
combined_df.replace(r'\|', ' ', regex=True, inplace=True) # pipe
combined_df.replace(r'\"', ' ', regex=True, inplace=True) # quotes

# Process the data to make sure the fields are compatible with the portal’s data type definition. 
# For numeric, make sure that all values can be recognized as a number. Missing values have to be encoded as "NaN". 
# For dates, the data has to be formatted as YYYY-MM-DD (you can also add a time to that - YYYY-MM-DD HH:MM:SS), and missing values have to be encoded as an empty text string ("").
# Check numeric columns

numeric_cols = ['CollectionDepth', 'CollectionReplicate', 'ResultsReplicate', 'Result']
for col in numeric_cols:
    try:
        combined_df[col].fillna('NaN')
    except:
        print('%s field does not exist for dataframe' % col)

# Cast data type for Result and MDL columns to numeric. Must be done here, not in the import data section
combined_df['Result'] = pd.to_numeric(combined_df['Result'], errors='coerce')
combined_df['MDL'] = pd.to_numeric(combined_df['MDL'], errors='coerce')

#### 6.2 Check latitude and longitude values.

In [15]:
def check_latitude(val):
    try:
        lat = float(val)
        return lat
    except TypeError:
        # a missing latitude value (and non-numeric values) should throw an error
        # missing values should be encoded as 'NaN' to define data type as numeric on open data portal
        return 'NaN'
    except ValueError:
        return 'NaN'

# Sometimes the Longitude gets entered as 119 instead of -119...
# Make sure Longitude value is negative and less than 10000 (could be projected)
# Check for missing and non-numeric values, replace with 'NaN'
def check_longitude(val):
    try:
        long = float(val)
        if 0. < long < 10000.0:
            val = -long
        return val
    except TypeError:
        # a missing latitude value (and non-numeric values) should throw an error
        # missing values should be encoded as 'NaN' to define data type as numeric on open data portal
        return 'NaN'
    except ValueError:
        return 'NaN'

combined_df['TargetLatitude'] = combined_df['TargetLatitude'].map(check_latitude).fillna('')
combined_df['TargetLongitude'] = combined_df['TargetLongitude'].map(check_longitude).fillna('')

#### 6.3 Drop records that do not have valid Result and MDL values
These records cannot be used even if we try to substitute the original value with 1/2 the MDL.

In [16]:
# Copy non-ND records that have a negative, null, or zero Result and a negative, null, or zero MDL value to a new dataframe
# These records will later be added to the rejected_records csv file output
# reject_result_df = combined_df[((pd.isna(combined_df['Result'])) | (combined_df['Result'] < 0) | (combined_df['Result'] == 0)) & ((pd.isna(combined_df['MDL'])) | (combined_df['MDL'] < 0) | (combined_df['Result'] == 0)) & (combined_df['ResultQualCode'] != 'ND')]
reject_result_df = combined_df[((combined_df['Result'].isnull()) | (combined_df['Result'] <= 0)) & ((combined_df['MDL'].isnull()) | (combined_df['MDL'] <= 0))]
reject_result_df['Comments'] = 'Result is null, negative, or zero; MDL is null, negative, or zero'
print('Count of unusable records to be dropped:', reject_result_df.shape[0])

# Drop the records from the dataset
combined_df = combined_df.drop(reject_result_df.index)

C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\2995662271.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reject_result_df['Comments'] = 'Result is null, negative, or zero; MDL is null, negative, or zero'


Count of unusable records to be dropped: 16443


#### 6.4 Drop replicate records

In [17]:
# Copy replicate records to a new dataframe
# These records will later be added to the rejected_records csv file output
replicate_df = combined_df[(combined_df['ResultsReplicate'] != 1) | (combined_df['CollectionReplicate'] != 1)]
replicate_df['Comments'] = 'Replicate data'
print('Count of replicate records to be dropped:', replicate_df.shape[0])

combined_df = combined_df.drop(replicate_df.index)

C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\578222717.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replicate_df['Comments'] = 'Replicate data'


Count of replicate records to be dropped: 43814


#### 6.5 Standardize unit values and drop unneeded records
There is inconsistency, mainly in the CEDEN database, with how the unit values are named. Later on, when calculating the geomeans, we will want to be able to group records by common unit values, so these values should match exactly.

In [18]:
# Rename units with abbreviations to have all capitalized letters
combined_df['Unit'] = combined_df['Unit'].replace('cfu/100mL', 'CFU/100 mL') 
combined_df['Unit'] = combined_df['Unit'].replace('mpn/100mL', 'MPN/100 mL') 

# Filter for specific units to be included in the dataset; copy all other records to new dataframe
units_keep = ['MPN/100 mL', 'CFU/100 mL', 'copies/100 mL']
reject_units_df = combined_df[~combined_df['Unit'].isin(units_keep)]
print('Count of unit records to filter out:', reject_units_df.shape[0])

Count of unit records to filter out: 4338


#### 6.6 Categorize records into unit groups based on the unit name
This is based on the assumption that results reported in MPN (most probable number) are equivalent to results reported in CFU (colony forming units). Result values reported in "copies/100 mL" are associated with ddPCR methods. They are not equivalent to either MPN/CFU and should be handled separately. 

In [19]:
# Assign a numeric value to each record based on the UnitName value
unit_map = { 'MPN/100 mL': 1, 'CFU/100 mL': 1, 'copies/100 mL': 2}
combined_df['UnitGroup'] = combined_df['Unit'].map(unit_map)  

combined_df.head()

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,SampleDateTime,UnitGroup
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,,0.0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:08:00,2015-07-07 11:08:00,1.0
1505327,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-16,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-16 00:00:00,1.0
1505326,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-09,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/09/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-09 00:00:00,1.0
1505325,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-12-09,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-12/09/2024,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,60.0,,10.0,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:27:00,2024-12-09 11:27:00,1.0
1505324,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-10-28,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-10/28/2024,Not Recorded,samplewater,Colilert-18,E. coli,MPN/100 mL,41.0,,10.0,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,E. coli,BeachWatch,,,,,,,,,,,11:27:00,2024-10-28 11:27:00,1.0


### 7. Add Datum column to the dataset
The data quality estimator tool (used in Step 8) requires the Datum field. This field is not included with the BeachWatch and CEDEN datasets by default, so we must get it from another CEDEN table and then join the values to the working dataset.

In [20]:
# Define a function used to get all records from the CEDEN table with datum data
def get_datum_data():
    try:
        sql = "SELECT StationCode, Datum FROM %s ;" % CEDEN_SITE_DATUM_TABLE
        cnxn = pyodbc.connect(Driver='SQL Server', Server=CEDEN_SERVER1, uid=CEDEN_UID, pwd=CEDEN_PWD)
        df = pd.read_sql(sql, cnxn)
        return df
    except:
        print("Couldn't connect to %s." % CEDEN_SERVER1)

datum_df = get_datum_data()
datum_df.head()

C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\881178188.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, cnxn)


,StationCode,Datum
0,000DC03xx,NAD83
1,000EWRRRx,NAD83
2,000POSR3,WGS84
3,000TCC204,NAD83
4,01_BPT_3,WGS84


In [21]:
# Join the datum data to the combined dataset on common StationCode IDs
data_df = pd.merge(combined_df, datum_df, on='StationCode', how='left')

# Fill empty datum values with 'NR'. This is an important step for the data quality estimator, used later
data_df = data_df.fillna(value={'Datum': 'NR'})

data_df.head()

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,SampleDateTime,UnitGroup,Datum
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,,0.0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:08:00,2015-07-07 11:08:00,1.0,NR
1,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-16,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-16 00:00:00,1.0,NR
2,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-09,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/09/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-09 00:00:00,1.0,NR
3,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-12-09,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-12/09/2024,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,60.0,,10.0,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:27:00,2024-12-09 11:27:00,1.0,NR
4,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-10-28,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-10/28/2024,Not Recorded,samplewater,Colilert-18,E. coli,MPN/100 mL,41.0,,10.0,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,E. coli,BeachWatch,,,,,,,,,,,11:27:00,2024-10-28 11:27:00,1.0,NR


### 8. Add a RegionNumber column to the dataset
This is a requested column to identify the Regional Board area where the site is located. We have to get data from another CEDEN stations table and join it to this dataset. This CEDEN table is a different table than the one used in Step 7. Unfortunately, the RB number values from this table are not complete. There will be some null values and other non-standard values in the dataset.

In [22]:
# Define a function that gets all records from the CEDEN station table, used to join region values.
def get_ceden_site_data():
    cnxn = pyodbc.connect(Driver='SQL Server', Server=CEDEN_SERVER1, uid=CEDEN_UID, pwd=CEDEN_PWD)
    sql = "SELECT StationLUCode, rb_number FROM %s" % CEDEN_SITE_TABLE
    df = pd.read_sql_query(sql, cnxn)
    return df

site_data = get_ceden_site_data()
site_data.head()

C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\1868677130.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, cnxn)


,StationLUCode,rb_number
0,000BBC003,0
1,000BBC008,0
2,000DC03xx,OOS
3,000EWK001,OOS
4,000EWRRRx,OOS


In [23]:
# Join the Region number to the combined dataset
data_df = data_df.merge(site_data, how='left', left_on='StationCode', right_on='StationLUCode')
data_df = data_df.rename(columns={'rb_number': 'RegionNumber'})

data_df.head()

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,SampleDateTime,UnitGroup,Datum,StationLUCode,RegionNumber
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,,0.0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:08:00,2015-07-07 11:08:00,1.0,NR,B-25,4
1,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-16,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-16 00:00:00,1.0,NR,MB-160,9
2,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-09,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/09/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-09 00:00:00,1.0,NR,MB-160,9
3,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-12-09,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-12/09/2024,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,60.0,,10.0,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:27:00,2024-12-09 11:27:00,1.0,NR,San Gregorio Creek,2
4,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-10-28,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-10/28/2024,Not Recorded,samplewater,Colilert-18,E. coli,MPN/100 mL,41.0,,10.0,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Record

### 9. Add data quality columns to the dataset
The OIMA data quality estimator tool adds two columns, DataQuality and DataQualityIndicator, to the dataset.

DataQuality: Describes the overall quality of the record by taking the QACode, ResulualQACode, ComplicanceCode, BatchVerificationCode, and special circumstances into account to assign it to one of the following categories: Passed, Some review needed, Spatial accuracy unknown, Extensive review needed, Unknown data quality, Reject record, Error in data, Metadata. The assignments and categories are provisional. A working explanation of the data quality ranking can be found this Google Doc file: https://docs.google.com/spreadsheets/d/1q-tGulvO9jyT2dR9GGROdy89z3W6xulYaci5-ezWAe0/edit?usp=sharing

DataQualityIndicator - Explains the reason for the DataQuality value by indicating which quality assurance check the data did not pass (e.g. BatchVerificationCode, ResultQACode, etc.).

The function "add_data_quality" used to add these two columns is imported into this notebook from another Python script file (below).

The code for the data quality estimator is hosted on GitHub here: https://github.com/mmtang/data-quality-estimator.
- The function *add_data_quality*: https://github.com/mmtang/data-quality-estimator/blob/master/data_quality.py
- The dictionaries for QACodes, ResultQualCodes, ComplianceCodes, etc. and their associated data quality values: https://github.com/mmtang/data-quality-estimator/blob/master/dq_constants.py

In [24]:
# Import Python file with the data quality estimator functions
import sys
sys.path.append('../data-quality-estimator')  # Path contains data_quality_utils.py

import data_quality

In [25]:
# Add the DataQuality and DataQualityIndicator columns
data_df = data_quality.add_data_quality(data_df, 'chemistry')

data_df.head()

VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a valid key in QA_Code_list
VFIRL not a 

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,SampleDateTime,UnitGroup,Datum,StationLUCode,RegionNumber,DataQuality,DataQualityIndicator
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,,0.0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:08:00,2015-07-07 11:08:00,1.0,NR,B-25,4,Unknown data quality,BatchVerification:NR; QACode:NR
1,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-16,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-16 00:00:00,1.0,NR,MB-160,9,Unknown data quality,BatchVerification:NR; QACode:NR
2,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-09,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/09/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-09 00:00:00,1.0,NR,MB-160,9,Unknown data quality,BatchVerification:NR; QACode:NR
3,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-12-09,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-12/09/2024,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,60.0,,10.0,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:27:00,2024-12-09 11:27:00,1.0,NR,San Gregorio Creek,2,Unknown data quality,BatchVerification:NR; QACode:NR
4,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-10-28,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-10/28/2024,Not Recorded,samplewater,Coliler

### 10. Drop records with a DataQuality score of "Reject record" or "Metadata"

In [26]:
# Copy records with a DataQuality score of 'Reject record' or 'MetaData to a new dataframe
# These records will later be added to the rejected_records csv file output
dq_filter = ['Reject record', 'MetaData']
reject_dq_df = data_df[data_df['DataQuality'].isin(dq_filter)]

# Drop these records from the dataset
data_df = data_df[~data_df['DataQuality'].isin(dq_filter)]

data_df.head()

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,SampleDateTime,UnitGroup,Datum,StationLUCode,RegionNumber,DataQuality,DataQualityIndicator
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,,0.0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:08:00,2015-07-07 11:08:00,1.0,NR,B-25,4,Unknown data quality,BatchVerification:NR; QACode:NR
1,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-16,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-16 00:00:00,1.0,NR,MB-160,9,Unknown data quality,BatchVerification:NR; QACode:NR
2,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-09,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/09/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-09 00:00:00,1.0,NR,MB-160,9,Unknown data quality,BatchVerification:NR; QACode:NR
3,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-12-09,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-12/09/2024,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,60.0,,10.0,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:27:00,2024-12-09 11:27:00,1.0,NR,San Gregorio Creek,2,Unknown data quality,BatchVerification:NR; QACode:NR
4,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-10-28,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-10/28/2024,Not Recorded,samplewater,Coliler

### 11. Drop records with sample dates before 1950
Some records with sample year values before the year 1950 are not being flagged. Drop these records and add them to the rejected_records csv file output.

In [27]:
reject_year_df = data_df[data_df['SampleDate'].dt.year < 1950]
reject_year_df['Comments'] = 'Flagged sample year'
print('Count of records to be dropped:', reject_year_df.shape[0])

# Drop the records from the dataset
data_df = data_df.drop(reject_year_df.index)

C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\66904717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reject_year_df['Comments'] = 'Flagged sample year'


Count of records to be dropped: 1545


### 12. Clean null values
For compatability with the open data portal

In [28]:
# We have to make a distinction between None, 'None', and ''
# 'None' and '' are used specifically in the datasets, but None gets translated to 'None' unless we replace it with '' explicitly
data_df.fillna('')

data_df.head()

C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\1186387314.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_df.fillna('')


,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,SampleDateTime,UnitGroup,Datum,StationLUCode,RegionNumber,DataQuality,DataQualityIndicator
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,,0.0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:08:00,2015-07-07 11:08:00,1.0,NR,B-25,4,Unknown data quality,BatchVerification:NR; QACode:NR
1,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-16,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-16 00:00:00,1.0,NR,MB-160,9,Unknown data quality,BatchVerification:NR; QACode:NR
2,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-09,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/09/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-09 00:00:00,1.0,NR,MB-160,9,Unknown data quality,BatchVerification:NR; QACode:NR
3,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-12-09,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-12/09/2024,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,60.0,,10.0,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:27:00,2024-12-09 11:27:00,1.0,NR,San Gregorio Creek,2,Unknown data quality,BatchVerification:NR; QACode:NR
4,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-10-28,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-10/28/2024,Not Recorded,samplewater,Coliler

### 13. Export a CSV file of all the dropped records. This includes:

- Step 5: Dropped duplicate records (duplicates_df)
- Step 6.3: Dropped records with unusable Result and MDL values (reject_result_df)
- Step 6.4: Dropped replicate records (replicate_df)
- Step 6.5: Dropped records with unit values we are not using (reject_units_df)
- Step 10: Dropped data quality records (reject_dq_df)
- Step 11: Dropped records with sample years before 1970, missed QA records (reject_year_df)

In [29]:
# Define fields to be included in file export
reject_export_fields = [
    'Program',
    'ParentProject',
    'Project',
    'StationName',
    'StationCode',
    'SampleDate',
    'CollectionTime',
    'LocationCode',
    'CollectionDepth',
    'UnitCollectionDepth',
    'SampleTypeCode',
    'CollectionReplicate',
    'ResultsReplicate',
    'LabBatch',
    'LabSampleID',
    'MatrixName',
    'MethodName',
    'Analyte',
    'Unit',
    'Result',
    'Observation',
    'MDL',
    'RL',
    'ResultQualCode',
    'QACode',
    'BatchVerification',
    'ComplianceCode',
    'SampleComments',
    'CollectionComments',
    'ResultsComments',
    'BatchComments',
    'EventCode',
    'ProtocolCode',
    'SampleAgency',
    'GroupSamples',
    'CollectionMethodName',
    'TargetLatitude',
    'TargetLongitude',
    'CollectionDeviceDescription',
    'CalibrationDate',
    'PositionWaterColumn',
    'PrepPreservationName',
    'PrepPreservationDate',
    'DigestExtractMethod',
    'DigestExtractDate',
    'AnalysisDate',
    'DilutionFactor',
    'ExpectedValue',
    'LabAgency',
    'SubmittingAgency',
    'SubmissionCode',
    'OccupationMethod',
    'StartingBank',
    'DistanceFromBank',
    'UnitDistanceFromBank',
    'StreamWidth',
    'UnitStreamWidth',
    'StationWaterDepth',
    'UnitStationWaterDepth',
    'HydroMod',
    'HydroModLoc',
    'LocationDetailWQComments',
    'ChannelWidth',
    'UpstreamLength',
    'DownStreamLength',
    'TotalReach',
    'LocationDetailBAComments',
    'SampleID',
    'DW_AnalyteName',
    'UnitGroup',
    'Datum',
    'DataSource',
    'SampleDateTime',
    'RegionNumber',
    'DataQuality',
    'DataQualityIndicator',
    'Comments'
]

# Merge all dataframes into a single dataframe
all_dropped_records_df = pd.concat([duplicates_df, reject_result_df, replicate_df, reject_units_df, reject_dq_df, reject_year_df], ignore_index=True)
all_dropped_records_df = all_dropped_records_df[reject_export_fields]

all_dropped_records_df.head()

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,HydroMod,HydroModLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,UnitGroup,Datum,DataSource,SampleDateTime,RegionNumber,DataQuality,DataQualityIndicator,Comments
0,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,PL-050-non-accessible or restricted access sho...,PL-050,2015-07-16,1900-01-01 10:41:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2015,Not Recorded,samplewater,SM 9222 B,"Coliform, Total",MPN/100 mL,20.0,,2,2,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.6794,-117.248,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",NaN,NaN,BeachWatch,2015-07-16 10:41:00,NaN,NaN,NaN,Duplicate record
1,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"PL-080-Ocean Beach, San Diego",PL-080,2015-07-22,1900-01-01 08:38:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/22/2015,Not Recorded,samplewater,SM 9222 B,"Coliform, Total",MPN/100 mL,60.0,,2,2,=,J,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7326,-117.257,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",NaN,NaN,BeachWatch,2015-07-22 08:38:00,NaN,NaN,NaN,Duplicate record
2,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"PL-090-Ocean Beach, San Diego",PL-090,2015-07-16,1900-01-01 08:59:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2015,Not Recorded,samplewater,SM 9222 B,"Coliform, Fecal",MPN/100 mL,8.0,,2,2,=,J,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7467,-117.255,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Fecal",NaN,NaN,BeachWatch,2015-07-16 08:59:00,NaN,NaN,NaN,Duplicate record
3,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,PL-040-non-accessible or restricted access sho...,PL-040,2015-07-16,1900-01-01 10:22:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2015,Not Recorded,samplewater,SM 9222 B,"Coliform, Fecal",MPN/100 mL,2.0,,2,2,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.6653,-117.243,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Fecal",NaN,NaN,BeachWatch,2015-07-16 10:22:00,NaN,NaN,NaN,Duplicate record
4,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,PL-040-non-accessible or restricted access sho...,PL-040,2015-07-22,1900-01-01 09:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/22/2015,Not Recorded,samplewater,SM 9222 B,Enterococcus,MPN/100 mL,2.0,,2,2,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.6653,-117.243,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,Enterococcus,NaN,NaN,BeachWatch,2015-07-22 09:00:00,NaN,NaN,NaN,Duplicate record


In [30]:
# Export all rejected records as a CSV file
all_dropped_records_df.to_csv('SafeToSwim_rejected_records.csv', index=False)

### 13. Handle non-detect (ND) records and assign substitute Result values
If a record is flagged as non-detect (ResultQualCode == 'ND'), substitute the Result value with either half the original Result value (if the Result > 0) or half the MDL (if the Result <= 0 or Result is null).

Also substitute half the MDL for records that are not flagged as non-detect but for some reason have a zero, null, or negative Result value. There shouldn't be very many (if any) of these records at this point, but I've left the code here just in case any slip through.

In [31]:
# Define a function for assigning substitute Result values
def subResult(row):
    if (row['ResultQualCode'] == 'ND'):
        if (row['Result'] > 0):
            return pd.Series([(0.5 * row['Result']), 'Nondetect: result substituted with half the result value'])
        elif (row['MDL'] > 0):
            return pd.Series([(0.5 * row['MDL']), 'Nondetect: result substituted with half the MDL'])
        else:
            return pd.Series([row['Result'], 'No substitution'])
    elif ((row['Result'] == 0) or (pd.isna(row['Result'])) or (row['Result'] < 0)):
        if (row['MDL'] > 0):
            return pd.Series([(0.5 * row['MDL']), 'Result substituted with half the MDL'])
        else:
            return pd.Series([row['Result'], 'No substitution'])
    else:
        return pd.Series([row['Result'], 'No substitution'])

# Apply the function to the entire dataframe and save the subbed and non-subbed Result values to a new dataframe
sub_values = data_df.apply(lambda x: subResult(x), axis=1)

# Copy over the values and comments to the original dataframe as a new column "ResultSub". The original "Result" column is left untouched for reference.
data_df['ResultSub'], data_df['ResultSubComments'] = sub_values[0], sub_values[1]

data_df.head()

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,SampleDateTime,UnitGroup,Datum,StationLUCode,RegionNumber,DataQuality,DataQualityIndicator,ResultSub,ResultSubComments
0,BeachWatch,BeachWatch_Long Beach City County,BeachWatch_Long Beach City County,"B-25-Long Beach, Long Beach City",B-25,2015-07-07,1900-01-01 11:08:00,SurfZone,-88.0,NR,Grab,1,1,CLBHHS-07/07/2015,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,10.0,,0.0,0,<,NR,NR,NR,,,,,wq,Not Recorded,CLBHHS,,Water_Grab,33.7712,-118.134,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CLBHHS,CLBHHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:08:00,2015-07-07 11:08:00,1.0,NR,B-25,4,Unknown data quality,BatchVerification:NR; QACode:NR,10.0,No substitution
1,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-16,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/16/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-16 00:00:00,1.0,NR,MB-160,9,Unknown data quality,BatchVerification:NR; QACode:NR,20.0,No substitution
2,BeachWatch,BeachWatch_San Diego County,BeachWatch_San Diego County,"MB-160-Mission Bay, Bahia Point, San Diego",MB-160,2002-07-09,1900-01-01 00:00:00,SurfZone,-88.0,NR,Grab,1,1,CSDDEH-07/09/2002,Not Recorded,samplewater,SM 9221 E,"Coliform, Total",MPN/100 mL,20.0,,10.0,10,<,NR,NR,NR,,,,,wq,Not Recorded,CSDDEH,,Water_Grab,32.7762,-117.246,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,CSDDEH,CSDDEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,00:00:00,2002-07-09 00:00:00,1.0,NR,MB-160,9,Unknown data quality,BatchVerification:NR; QACode:NR,20.0,No substitution
3,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Creek,2024-12-09,1900-01-01 11:27:00,SurfZone,-88.0,NR,Grab,1,1,HDCSMEH-12/09/2024,Not Recorded,samplewater,Colilert-18,"Coliform, Total",MPN/100 mL,60.0,,10.0,10,=,NR,NR,NR,,,,,wq,Not Recorded,HDCSMEH,,Water_Grab,37.321,-122.403,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,0.0,0,HDCSMEH,HDCSMEH,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,11:27:00,2024-12-09 11:27:00,1.0,NR,San Gregorio Creek,2,Unknown data quality,BatchVerification:NR; QACode:NR,60.0,No substitution
4,BeachWatch,BeachWatch_San Mateo County,BeachWatch_San Mateo County,"San Gregorio Creek-San Gregorio State Beach, S...",San Gregorio Cre

### 14. Calculate the geometric mean values

#### 14.1 Required data prep before calculating the geometric mean

In [32]:
# Ensure that SampleDateTime values are cast as datetime objects
data_df['SampleDateTime'] = data_df['SampleDateTime'].astype('datetime64[ns]')

# Set SampleDateTime as the index. This is more efficient for the grouping operations
data_df.set_index('SampleDateTime', inplace=True) 

# Drop records that have a null/NaT SampleDate value. As of 6-18-24, this is just one record.
data_df = data_df.loc[data_df.index.notnull()] 

# Sort records based on ascending SampleDateTime. A bit counterintuitive, but this is the setup for calculating 
# the rolling geometric starting from the most recent sample date working backwards using the rolling function
data_df.sort_index(ascending=True, inplace=True) 

data_df.head()

,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Analyte,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,UnitGroup,Datum,StationLUCode,RegionNumber,DataQuality,DataQualityIndicator,ResultSub,ResultSubComments
SampleDateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1969-04-15 23:59:00,BeachWatch,BeachWatch_Santa Cruz County,BeachWatch_Santa Cruz County,"O099-Hidden Beach, Santa Cruz",O099,1969-04-15,1900-01-01 23:59:00,SurfZone,-88.0,NR,Grab,1,1,SCCEHS-04/15/1969,Not Recorded,samplewater,SM 9222 B,"Coliform, Total",CFU/100 mL,29.9,,0.0,0,=,NR,NR,NR,,,,,wq,Not Recorded,SCCEHS,,Water_Grab,36.9571,-121.889,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,SCCEHS,SCCEHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,23:59:00,1.0,NR,O099,3,Unknown data quality,BatchVerification:NR; QACode:NR,29.9,No substitution
1969-04-15 23:59:00,BeachWatch,BeachWatch_Santa Cruz County,BeachWatch_Santa Cruz County,"O510-Lighthouse Beach, Santa Cruz",O510,1969-04-15,1900-01-01 23:59:00,SurfZone,-88.0,NR,Grab,1,1,SCCEHS-04/15/1969,Not Recorded,samplewater,SM 9222 B,"Coliform, Total",CFU/100 mL,91.0,,0.0,0,=,NR,NR,NR,,,,,wq,Not Recorded,SCCEHS,,Water_Grab,36.9517,-122.029,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,SCCEHS,SCCEHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,23:59:00,1.0,NR,O510,3,Unknown data quality,BatchVerification:NR; QACode:NR,91.0,No substitution
1969-04-15 23:59:00,BeachWatch,BeachWatch_Santa Cruz County,BeachWatch_Santa Cruz County,"O320-Pleasure Point Beach, Santa Cruz",O320,1969-04-15,1900-01-01 23:59:00,SurfZone,-88.0,NR,Grab,1,1,SCCEHS-04/15/1969,Not Recorded,samplewater,SM 9222 B,"Coliform, Total",CFU/100 mL,91.0,,0.0,0,=,NR,NR,NR,,,,,wq,Not Recorded,SCCEHS,,Water_Grab,36.9587,-121.967,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,SCCEHS,SCCEHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,23:59:00,1.0,NR,O320,3,Unknown data quality,BatchVerification:NR; QACode:NR,91.0,No substitution
1969-04-15 23:59:00,BeachWatch,BeachWatch_Santa Cruz County,BeachWatch_Santa Cruz County,"O520-Mitchell's Cove Beach, Santa Cruz",O520,1969-04-15,1900-01-01 23:59:00,SurfZone,-88.0,NR,Grab,1,1,SCCEHS-04/15/1969,Not Recorded,samplewater,SM 9222 B,"Coliform, Total",CFU/100 mL,230.0,,0.0,0,=,NR,NR,NR,,,,,wq,Not Recorded,SCCEHS,,Water_Grab,36.9526,-122.041,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,SCCEHS,SCCEHS,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Total",BeachWatch,,,,,,,,,,,23:59:00,1.0,NR,O520,3,Unknown data quality,BatchVerification:NR; QACode:NR,230.0,No substitution
1969-04-15 23:59:00,BeachWatch,BeachWatch_Santa Cruz County,BeachWatch_Santa Cruz County,"O010-Pajaro Dunes Beach, Santa Cruz",O010,196

#### 14.2 Group records and calculate the geometric mean
This code block adds six new columns:

- 30DayCutoffDate: The cutoff date used to determine which results fall within the 30 day range for calculating the rolling geometric mean.
- 30DayGeoMean: The rolling geometric mean value looking back 30 days from the recorded sample date.
- 30DayCount: The number of distinct sample result values included in the 30 day date range and used in the geometric mean calculation.
- 6WeekCutoffDate: The cutoff date used to determine which results fall within the 6 week range for calculating the rolling geometric mean.
- 6WeekGeoMean: The rolling geometric mean value looking back 6 weeks (42 days) from the recorded sample date.
- 6WeekCount: The number of distinct sample result values included in the 6 week date range and used in the geometric mean calculation.

In [33]:
# Function for calculating and adding the geometric mean columns to a grouped dataframe
def process_group(df):
    # Nested function for calculating the geometric mean         
    def calculateGeometricMean(x):
        # Need to group records again or else the Result values are duplicated in the calculation
        x = x.groupby(level=0).mean()
        g_value = gmean(x, nan_policy='omit') # gmean is a SciPy function
        return g_value

    # It is not recommended to mutate the object we're iterating on, thus the copy:
    # https://pandas.pydata.org/docs/user_guide/gotchas.html#mutating-with-user-defined-function-udf-methods
    df = df.copy() 

    # Calculate 30 day rolling geomean
    df['30DayGeoMean'] = df['ResultSub'].rolling(window='30D', min_periods=1, closed='both').apply(calculateGeometricMean).round(3) 
    df['30DayCount'] = df['ResultSub'].rolling(window='30D', min_periods=1, closed='both').apply(lambda x: len(x.groupby(level=0)))

    # Calculate 6 week rolling geomean. 6 weeks = 42 days
    df['6WeekGeoMean'] = df['ResultSub'].rolling(window='42D', min_periods=1, closed='both').apply(calculateGeometricMean).round(3)
    df['6WeekCount'] = df['ResultSub'].rolling(window='42D', min_periods=1, closed='both').apply(lambda x: len(x.groupby(level=0))) 

    # Drop duplicate records
    df = df.groupby(level=0).last()
    return df

# Calculate new geometric mean values for all FIB records based on the SampleDateTime index and common column values as defined in group_cols
# Set allow_duplicates=True to reinsert index columns into the dataframe and allow columns with the same name
group_cols = ['Analyte', 'StationCode', 'UnitGroup']
grouped_df = data_df.groupby(group_cols).apply(process_group).reset_index(allow_duplicates=True)

# Drop duplicate columns. There might be some duplicate columns leftover after the new geomean columns are inserted back into the dataframe
grouped_df = grouped_df.loc[:,~grouped_df.columns.duplicated()]

grouped_df.head()

C:\Users\mtang\AppData\Local\Temp\2\ipykernel_300\3220801032.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = data_df.groupby(group_cols).apply(process_group).reset_index(allow_duplicates=True)


,Analyte,StationCode,UnitGroup,SampleDateTime,Program,ParentProject,Project,StationName,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,SampleTypeCode,CollectionReplicate,ResultsReplicate,LabBatch,LabSampleID,MatrixName,MethodName,Unit,Result,Observation,MDL,RL,ResultQualCode,QACode,BatchVerification,ComplianceCode,SampleComments,CollectionComments,ResultsComments,BatchComments,EventCode,ProtocolCode,SampleAgency,GroupSamples,CollectionMethodName,TargetLatitude,TargetLongitude,CollectionDeviceDescription,CalibrationDate,PositionWaterColumn,PrepPreservationName,PrepPreservationDate,DigestExtractMethod,DigestExtractDate,AnalysisDate,DilutionFactor,ExpectedValue,LabAgency,SubmittingAgency,SubmissionCode,OccupationMethod,StartingBank,DistanceFromBank,UnitDistanceFromBank,StreamWidth,UnitStreamWidth,StationWaterDepth,UnitStationWaterDepth,Hydromod,HydromodLoc,LocationDetailWQComments,ChannelWidth,UpstreamLength,DownStreamLength,TotalReach,LocationDetailBAComments,SampleID,DW_AnalyteName,DataSource,HydroMod,HydroModLoc,isQA,ProgramCode,ParentProjectCode,ProjectCode,MatrixCode,AnalyteCode,FractionName,FractionCode,CollectionTimeOnly,Datum,StationLUCode,RegionNumber,DataQuality,DataQualityIndicator,ResultSub,ResultSubComments,30DayGeoMean,30DayCount,6WeekGeoMean,6WeekCount
0,"Coliform, Fecal",0,1.0,1998-03-02 08:00:00,BeachWatch,BeachWatch_Orange County,BeachWatch_Orange County,"0-Huntington State Beach, Orange",1998-03-02,1900-01-01 08:00:00,SurfZone,-88.0,NR,Grab,1,1,OC-03/02/1998,Not Recorded,samplewater,SM 9221 E,MPN/100 mL,700.0,,0.0,0,=,NR,NR,NR,,,,,wq,Not Recorded,OCEHD,,Water_Grab,33.6293,-117.96,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,OC,OC,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Fecal",BeachWatch,,,,,,,,,,,08:00:00,NR,0,8,Unknown data quality,BatchVerification:NR; QACode:NR,700.0,No substitution,700.000,1.0,700.000,1.0
1,"Coliform, Fecal",0,1.0,1998-03-03 08:00:00,BeachWatch,BeachWatch_Orange County,BeachWatch_Orange County,"0-Huntington State Beach, Orange",1998-03-03,1900-01-01 08:00:00,SurfZone,-88.0,NR,Grab,1,1,OC-03/03/1998,Not Recorded,samplewater,SM 9221 E,MPN/100 mL,230.0,,0.0,0,=,NR,NR,NR,,,,,wq,Not Recorded,OCEHD,,Water_Grab,33.6293,-117.96,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,OC,OC,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Fecal",BeachWatch,,,,,,,,,,,08:00:00,NR,0,8,Unknown data quality,BatchVerification:NR; QACode:NR,230.0,No substitution,401.248,2.0,401.248,2.0
2,"Coliform, Fecal",0,1.0,1998-03-04 08:00:00,BeachWatch,BeachWatch_Orange County,BeachWatch_Orange County,"0-Huntington State Beach, Orange",1998-03-04,1900-01-01 08:00:00,SurfZone,-88.0,NR,Grab,1,1,OC-03/04/1998,Not Recorded,samplewater,SM 9221 E,MPN/100 mL,80.0,,0.0,0,=,NR,NR,NR,,,,,wq,Not Recorded,OCEHD,,Water_Grab,33.6293,-117.96,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,OC,OC,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Fecal",BeachWatch,,,,,,,,,,,08:00:00,NR,0,8,Unknown data quality,BatchVerification:NR; QACode:NR,80.0,No substitution,234.408,3.0,234.408,3.0
3,"Coliform, Fecal",0,1.0,1998-03-13 08:00:00,BeachWatch,BeachWatch_Orange County,BeachWatch_Orange County,"0-Huntington State Beach, Orange",1998-03-13,1900-01-01 08:00:00,SurfZone,-88.0,NR,Grab,1,1,OC-03/13/1998,Not Recorded,samplewater,SM 9221 E,MPN/100 mL,40.0,,0.0,0,=,NR,NR,NR,,,,,wq,Not Recorded,OCEHD,,Water_Grab,33.6293,-117.96,Not Recorded,NaT,Not Recorded,Not Recorded,1950-01-01,Not Recorded,1950-01-01,1950-01-01,1.0,0,OC,OC,NR,,,,,,,,,,,,,,,,,Not Recorded,"Coliform, Fecal",BeachWatch,,,,,,,,,,,08:00:00,NR,0,8,Unknown data quality,BatchVerification:NR; QACode:NR,40.0,No substitution,150.659,4.0,150.659,4.0
4,"Coliform, Fecal",0,1.0,1998-03-17 08:00:00,BeachWatch,BeachWatch_Orange County,BeachWatch_Orange County,"0-Huntington State Beach, Orange",1998-03-17,1900-01-01 08:00:00,SurfZone,-88.0,NR,

In [34]:
# Add the cutoff dates for the geometric mean calculations - for reference/documentation mainly
print(grouped_df.dtypes)
grouped_df['30DayCutoffDate'] = grouped_df['SampleDateTime'] - timedelta(days=30)
grouped_df['6WeekCutoffDate'] = grouped_df['SampleDateTime'] - timedelta(days=41)

Analyte                      object
StationCode                  object
UnitGroup                   float64
SampleDateTime       datetime64[ns]
Program                      object
                          ...      
ResultSubComments            object
30DayGeoMean                float64
30DayCount                  float64
6WeekGeoMean                float64
6WeekCount                  float64
Length: 94, dtype: object


### 15. Export the geomean dataset as a CSV file

#### 15.1 Export the full dataset with all columns

In [35]:
all_fields = [
    'Program',
    'ParentProject',
    'Project',
    'StationName',
    'StationCode',
    'SampleDate',
    'CollectionTime',
    'LocationCode',
    'CollectionDepth',
    'UnitCollectionDepth',
    'SampleTypeCode',
    'CollectionReplicate',
    'ResultsReplicate',
    'LabBatch',
    'LabSampleID',
    'MatrixName',
    'MethodName',
    'Analyte',
    'Unit',
    'Result',
    'Observation',
    'MDL',
    'RL',
    'ResultQualCode',
    'QACode',
    'BatchVerification',
    'ComplianceCode',
    'SampleComments',
    'CollectionComments',
    'ResultsComments',
    'BatchComments',
    'EventCode',
    'ProtocolCode',
    'SampleAgency',
    'GroupSamples',
    'CollectionMethodName',
    'TargetLatitude',
    'TargetLongitude',
    'CollectionDeviceDescription',
    'CalibrationDate',
    'PositionWaterColumn',
    'PrepPreservationName',
    'PrepPreservationDate',
    'DigestExtractMethod',
    'DigestExtractDate',
    'AnalysisDate',
    'DilutionFactor',
    'ExpectedValue',
    'LabAgency',
    'SubmittingAgency',
    'SubmissionCode',
    'OccupationMethod',
    'StartingBank',
    'DistanceFromBank',
    'UnitDistanceFromBank',
    'StreamWidth',
    'UnitStreamWidth',
    'StationWaterDepth',
    'UnitStationWaterDepth',
    'HydroMod',
    'HydroModLoc',
    'LocationDetailWQComments',
    'ChannelWidth',
    'UpstreamLength',
    'DownStreamLength',
    'TotalReach',
    'LocationDetailBAComments',
    'SampleID',
    'DW_AnalyteName',
    #'UnitGroup',
    'Datum',
    #'CollectionTimeOnly',
    'DataSource',
    'SampleDateTime',
    'RegionNumber',
    'DataQuality',
    'DataQualityIndicator',
    'ResultSub',
    'ResultSubComments',
    #'ResultAvg',
    '30DayGeoMean',
    '30DayCount',
    '30DayCutoffDate',
    '6WeekGeoMean',
    '6WeekCount',
    '6WeekCutoffDate'
]

# Order columns
grouped_df_full = grouped_df[all_fields]

# Export dataframe as a CSV file
grouped_df_full.to_csv('SafeToSwim_geomeans_full.csv', index=False)

#### 15.2 Export the full dataset as multiple files
Export a version of the dataset for upload to the open data portal (data.ca.gov). The portal has a file size limit, so we will split the dataset into multiple files based on sample date.

In [36]:
data_before_2010 = grouped_df_full[grouped_df_full['SampleDate'] < '2010-01-01']
data_2010_2020 = grouped_df_full[(grouped_df_full['SampleDate'] >= '2010-01-01') & (grouped_df_full['SampleDate'] < '2020-01-01')]
data_2020_present = grouped_df_full[grouped_df_full['SampleDate'] >= '2020-01-01']

data_before_2010.to_csv('SafeToSwim_geomeans_before-2010.csv', index=False)
data_2010_2020.to_csv('SafeToSwim_geomeans_2010-2020.csv', index=False)
data_2020_present.to_csv('SafeToSwim_geomeans_2020-present.csv', index=False)

#### 15.2 Dataset with select columns (for testing)
Export an shortened version of the dataset (fewer columns) for testing. 

In [37]:
'''
test_fields = [
    'Program',
    'ParentProject',
    'Project',
    'StationName',
    'StationCode',
    'SampleDate',
    'CollectionTime',
    #'LocationCode',
    #'CollectionDepth',
    #'UnitCollectionDepth',
    #'SampleTypeCode',
    #'CollectionReplicate',
    #'ResultsReplicate',
    'LabBatch',
    #'LabSampleID',
    'MatrixName',
    'MethodName',
    'Analyte',
    'Unit',
    'Result',
    #'Observation',
    'MDL',
    'RL',
    'ResultQualCode',
    #'QACode',
    #'BatchVerification',
    #'ComplianceCode',
    #'SampleComments',
    #'CollectionComments',
    #'ResultsComments',
    #'BatchComments',
    #'EventCode',
    #'ProtocolCode',
    #'SampleAgency',
    #'GroupSamples',
    #'CollectionMethodName',
    #'TargetLatitude',
    #'TargetLongitude',
    #'CollectionDeviceDescription',
    #'CalibrationDate',
    #'PositionWaterColumn',
    #'PrepPreservationName',
    #'PrepPreservationDate',
    #'DigestExtractMethod',
    #'DigestExtractDate',
    #'AnalysisDate',
    #'DilutionFactor',
    #'ExpectedValue',
    #'LabAgency',
    #'SubmittingAgency',
    #'SubmissionCode',
    #'OccupationMethod',
    #'StartingBank',
    #'DistanceFromBank',
    #'UnitDistanceFromBank',
    #'StreamWidth',
    #'UnitStreamWidth',
    #'StationWaterDepth',
    #'UnitStationWaterDepth',
    #'HydroMod',
    #'HydroModLoc',
    #'LocationDetailWQComments',
    #'ChannelWidth',
    #'UpstreamLength',
    #'DownStreamLength',
    #'TotalReach',
    #'LocationDetailBAComments',
    #'SampleID',
    #'DW_AnalyteName',
    #'Datum',
    #'CollectionTimeOnly',
    'DataSource',
    'SampleDateTime',
    'RegionNumber',
    'DataQuality',
    'DataQualityIndicator',
    'ResultSub',
    'ResultSubComments',
    #'ResultAvg',
    '30DayGeoMean',
    '30DayCount',
    '30DayCutoffDate',
    '6WeekGeoMean',
    '6WeekCount',
    '6WeekCutoffDate'
]

# Order columns
grouped_df_test = grouped_df[test_fields]

# Export dataframe as a CSV file
grouped_df_test.to_csv('SafeToSwim_geomeans_short.csv', index=False)
'''

"\ntest_fields = [\n    'Program',\n    'ParentProject',\n    'Project',\n    'StationName',\n    'StationCode',\n    'SampleDate',\n    'CollectionTime',\n    #'LocationCode',\n    #'CollectionDepth',\n    #'UnitCollectionDepth',\n    #'SampleTypeCode',\n    #'CollectionReplicate',\n    #'ResultsReplicate',\n    'LabBatch',\n    #'LabSampleID',\n    'MatrixName',\n    'MethodName',\n    'Analyte',\n    'Unit',\n    'Result',\n    #'Observation',\n    'MDL',\n    'RL',\n    'ResultQualCode',\n    #'QACode',\n    #'BatchVerification',\n    #'ComplianceCode',\n    #'SampleComments',\n    #'CollectionComments',\n    #'ResultsComments',\n    #'BatchComments',\n    #'EventCode',\n    #'ProtocolCode',\n    #'SampleAgency',\n    #'GroupSamples',\n    #'CollectionMethodName',\n    #'TargetLatitude',\n    #'TargetLongitude',\n    #'CollectionDeviceDescription',\n    #'CalibrationDate',\n    #'PositionWaterColumn',\n    #'PrepPreservationName',\n    #'PrepPreservationDate',\n    #'DigestExtractM